In [1]:
# ruta local one drive

ruta ='/content/drive/MyDrive/Capstone Project/Modelo Final/'

In [2]:
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from pandas.tseries.offsets import MonthEnd

Mounted at /content/drive


In [3]:
# consumo en toneladas, gasolina 95, 98 , diesel A y B

path_consumo = ruta+'consumos-pp-ccaa-provincias.xlsx'
df = pd.read_excel(path_consumo, sheet_name='Consumos', skiprows=range(0,5))
df.fillna(0, inplace=True)
df=df[df['Provincia']=='Total']

df['Gasolina 95']=df['Gasolina 95 I.O.']
df['Gasolina 98']=df['Gasolina 98 I.O.']
df['Diesel A']=df['Gasóleo A ']
df['Diesel B']=df['Gasóleo B']
df['CCAA'] = df['CCAA'].replace('Total Nacional', 'España')

df.drop(columns=['Provincia','Gasolina 97 I.O.','Gasolina 95 I.O.','Gasolina 98 I.O.','Gasóleo A ','Gasóleo B','Gasóleo C','% bio en  gasóleos','% bio en  gasolinas','Fuelóleo BIA'], inplace=True)

month_map = {
    'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
    'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
    'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
}

df['mes_lower'] = df['Mes'].str.lower()
df['mes_num'] = df['mes_lower'].map(month_map)
df['Period'] = df['Año'].astype(str) + '-' + df['mes_num']

df = df.melt(
    id_vars=['Period', 'CCAA'],
    value_vars=['Gasolina 95','Gasolina 98','Diesel A','Diesel B'],
    var_name='Product',
    value_name='Tonnes'
)

df = (
    df[['Period', 'CCAA', 'Product', 'Tonnes']]
    .sort_values(['Period', 'CCAA', 'Product'])
    .reset_index(drop=True)
)

df['Period'] = pd.PeriodIndex(df['Period'], freq='M')

In [4]:
# integracion varible gdop, con transforamcion mensual

gdp_wide = pd.read_excel(
    ruta+'GDP_REGIONS_Quaterly.xlsx',
    sheet_name='PIB trim CCAA',
    skiprows=1,
    usecols=range(0,19)
)
gdp_wide.rename(columns={'Unnamed: 0': 'Period'}, inplace=True)

def trimestral_a_finmes(x):
    s = str(x)
    año, tr = s[:4], s[-2:]
    mes = {'01':'03','02':'06','03':'09','04':'12'}[tr]
    return pd.to_datetime(f"{año}-{mes}-01") + pd.offsets.MonthEnd(0)

gdp_wide['date'] = gdp_wide['Period'].apply(trimestral_a_finmes)
gdp_wide.set_index('date', inplace=True)
gdp_wide.drop(columns=['Period'], inplace=True)

gdp_monthly = (
    gdp_wide
    .resample('M')
    .interpolate(method='linear')
    .reset_index()
)

gdp_long = gdp_monthly.melt(
    id_vars=['date'],
    var_name='CCAA',
    value_name='gdp'
)

gdp_long['Period'] = gdp_long['date'].dt.to_period('M')

region_map = {
    'Andalucía':                   'Andalucía',
    'Aragón':                      'Aragón',
    'Asturias, Principado de':     'Asturias',
    'Balears, Illes':              'Baleares',
    'Canarias':                    'Canarias',
    'Cantabria':                   'Cantabria',
    'Castilla y León':             'Castilla y León',
    'Castilla - La Mancha':        'Castilla-La Mancha',
    'Cataluña':                    'Cataluña',
    'Comunitat Valenciana':        'Comunidad Valenciana',
    'Extremadura':                 'Extremadura',
    'Galicia':                     'Galicia',
    'Madrid, Comunidad de':        'Madrid',
    'Murcia, Región de':           'Murcia',
    'Navarra, Comunidad Foral de': 'Navarra',
    'País Vasco':                  'País Vasco',
    'Rioja, La':                   'La Rioja',
    'España':                      'España'
}

gdp_long['CCAA'] = gdp_long['CCAA'].replace(region_map)

ccaa_gdp = df.merge(
    gdp_long[['Period','CCAA','gdp']],
    on=['Period','CCAA'],
    how='left'
)

ccaa_gdp['Period'] = pd.PeriodIndex(ccaa_gdp['Period'], freq='M')

faltan = set(df['CCAA']) - set(gdp_long['CCAA'])
print("Regiones en ccaa sin dato de GDP:", faltan)

# es normal que falten ceuta y melilla, quedan fuera del scope del proyecto

Regiones en ccaa sin dato de GDP: {'Melilla', 'Ceuta'}


In [5]:
# integacion de los precios de los productos gasolina 95, 98 , diesel A y B

price_to_ccaa  = {
    'Toda España':              'España',
    'Andalucia':                'Andalucía',
    'Aragon':                   'Aragón',
    'Asturias':                 'Asturias',
    'Baleares':                 'Baleares',
    'Canarias':                 'Canarias',
    'Cantabria':                'Cantabria',
    'Castilla la Mancha':       'Castilla-La Mancha',
    'Castilla y León':          'Castilla y León',
    'Cataluña':                 'Cataluña',
    'Comunidad Valencia':       'Comunidad Valenciana',
    'Extremadura':              'Extremadura',
    'Galicia':                  'Galicia',
    'Madrid':                   'Madrid',
    'Murcia':                   'Murcia',
    'Navarra':                  'Navarra',
    'País Vasco':               'País Vasco',
    'Rioja (La)':               'La Rioja',
    'Ceuta':                    'Ceuta',
    'Melilla':                  'Melilla'
}


def prep_price_df(df_price: pd.DataFrame, product_name: str) -> pd.DataFrame:
    df = df_price.reset_index()
    drop_cols = [c for c in df.columns if c.lower().startswith('toda espa')]
    df = df.drop(columns=drop_cols, errors='ignore')
    df = df.rename(columns=price_to_ccaa)
    df_long = df.melt(id_vars=['Fecha'],
                      var_name='CCAA',
                      value_name='Precio')
    df_long['Period'] = pd.to_datetime(df_long['Fecha']).dt.to_period('M')
    df_long['Product'] = product_name
    return df_long[['Period', 'CCAA', 'Product', 'Precio']]

path_diesel_A = ruta+'Prices_Gasoleo_A.xlsx'
precios_diesel_A=pd.read_excel(path_diesel_A, index_col='Fecha').fillna(0)

path_diesel_B = ruta+'Prices_Gasoleo_B.xlsx'
precios_diesel_B=pd.read_excel(path_diesel_B, index_col='Fecha').fillna(0)

path_gasolina_95=ruta+'Prices_Gasoline_95.xlsx'
precios_gasolina_95=pd.read_excel(path_gasolina_95, index_col='Fecha').fillna(0)

path_gasolina_98=ruta+'Prices_Gasoline_98.xlsx'
precios_gasolina_98=pd.read_excel(path_gasolina_98, index_col='Fecha').fillna(0)

gas_long_95 = prep_price_df(precios_gasolina_95,   'Gasolina 95')

gas_long_98 = prep_price_df(precios_gasolina_98,   'Gasolina 98')

die_long_A = prep_price_df(precios_diesel_A,     'Diesel A')

die_long_B = prep_price_df(precios_diesel_B,     'Diesel B')


price_long = pd.concat([gas_long_95, gas_long_98,die_long_A,die_long_B],  ignore_index=True)

ccaa_gdp_precio = ccaa_gdp.merge(
    price_long,
    on=['Period','CCAA','Product'],
    how='left'
)

simple_mean = (
    ccaa_gdp_precio[ccaa_gdp_precio['CCAA'] != 'España']
    .groupby(['Period', 'Product'])['Precio']
    .mean()
    .rename('MeanPrecio')
    .reset_index()
)

ccaa_gdp_precio = ccaa_gdp_precio.merge(
    simple_mean,
    on=['Period', 'Product'],
    how='left'
)

mask_spain = ccaa_gdp_precio['CCAA'] == 'España'
ccaa_gdp_precio.loc[mask_spain, 'Precio'] = ccaa_gdp_precio.loc[mask_spain, 'MeanPrecio']
ccaa_gdp_precio = ccaa_gdp_precio.drop(columns=['MeanPrecio'])

ccaa_gdp_precio['Period'] = ccaa_gdp_precio['Period'].dt.to_timestamp(freq='M')

In [6]:
# integracion de las matriculaciones

matriculaciones_path = ruta+'matriculaciones.xlsx'
matriculaciones=pd.read_excel(matriculaciones_path)

matriculaciones['Period'] = (
    pd.to_datetime(
        dict(year=matriculaciones['Año'],
             month=matriculaciones['Mes'],
             day=1)
    )
    + MonthEnd(1)
)

matriculaciones = matriculaciones.rename(columns={'Comunidad': 'CCAA'})

productos = ['Gasolina 95', 'Gasolina 98', 'Diesel A', 'Diesel B']

mat_long = (
    matriculaciones
    .melt(
        id_vars=['Period', 'CCAA'],
        value_vars=productos,
        var_name='Product',
        value_name='matriculaciones'
    )
)

ccaa_gdp_precio_matriculaciones = (
    ccaa_gdp_precio
    .merge(
        mat_long,
        on=['Period', 'CCAA', 'Product'],
        how='left'
    )
)



In [7]:
# --- 1. Asegúrate de que 'Period' en tu df base es datetime ---
ccaa_gdp_precio_matriculaciones['Period'] = pd.to_datetime(
    ccaa_gdp_precio_matriculaciones['Period']
)

# --- 2. Extraigo un único GDP por (Period, CCAA) para evitar duplicados en el merge ---
df_gdp = (
    ccaa_gdp_precio_matriculaciones[['Period', 'CCAA', 'gdp','Precio']]
    .drop_duplicates()
    .reset_index(drop=True)
)

# --- 3. Leo y formateo el fichero HVO (solo España) ---
hvo = pd.read_excel(ruta + 'HVO España.xlsx')
hvo['CCAA'] = 'España'
hvo['Period'] = (
    pd.to_datetime(hvo['Año'].astype(str) + '-' +
                   hvo['Mes'].astype(str) + '-01')
    + pd.offsets.MonthEnd(0)
)

# --- 4. Construyo las filas HVO para España y traigo solo el GDP ---
df_hvo_es = hvo[['Period', 'CCAA']].copy()
df_hvo_es['Product'] = 'HVO'
df_hvo_es['Tonnes'] = hvo['HVO']
df_hvo_es['matriculaciones'] = 0
df_hvo_es['Precio']=0

df_hvo_es = df_hvo_es.merge(
    df_gdp,
    on=['Period', 'CCAA','Precio'],
    how='left'
)

# --- 5. Construyo las filas HVO para las demás CCAA (Tonnes = 0) ---
otras_ccaa = [
    c for c in ccaa_gdp_precio_matriculaciones['CCAA'].unique()
    if c != 'España'
]
periodos = df_hvo_es['Period'].unique()
rows = []
for periodo in periodos:
    for ccaa in otras_ccaa:
        rows.append({'Period': periodo, 'CCAA': ccaa})

df_hvo_otras = pd.DataFrame(rows)
df_hvo_otras['Product'] = 'HVO'
df_hvo_otras['Tonnes'] = 0
df_hvo_otras['matriculaciones'] = 0
df_hvo_otras['Precio']=0

df_hvo_otras = df_hvo_otras.merge(
    df_gdp,
    on=['Period', 'CCAA','Precio'],
    how='left'
)

# --- 6. Concateno todo y elimino la columna 'Precio' si existiera ---
df_final = pd.concat(
    [ccaa_gdp_precio_matriculaciones, df_hvo_es, df_hvo_otras],
    ignore_index=True
)
#if 'Precio' in df_final.columns:
#    df_final = df_final.drop(columns=['Precio'])

df_final = (
    df_final
    .sort_values(['Period', 'CCAA', 'Product'])
    .reset_index(drop=True)
)

df_final.fillna(0, inplace=True)

In [8]:
# estructura df final

df_final['Period'] = pd.to_datetime(df_final['Period'])
df_final['Period'] = df_final['Period'] + pd.offsets.MonthEnd(0)
df_final['Period'] = df_final['Period'].dt.strftime('%Y-%m-%d')
df_final = df_final.sort_values(['Period', 'CCAA', 'Product']).reset_index(drop=True)
#df_final=df_final.dropna()

In [9]:
df_final[(df_final['CCAA']=='España')].to_excel('df_final_España.xlsx', index=False)
from google.colab import files
files.download('df_final_España.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# descarga plano completo

df_final.to_excel('df_final.xlsx', index=False)

from google.colab import files
files.download('df_final.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>